# Bare BCS for surrogate construction

This notebook constructs a surrogate with arbitrary basis functions for Genz functions and gives the normalized root mean square error between the surrogate and the actual function. Both the Genz function and the basis are defined on [-1,1].

In [1]:
import numpy as np
import math  
from scipy.stats import qmc
import pandas as pd

import PyUQTk.pce as uqtkpce
import PyUQTk.PyPCE.pce_tools as pce_tools
from PyUQTk.utils.func import *


import matplotlib.pyplot as plt
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.datasets import make_sparse_coded_signal

PyMC is required for some of the MCMC postprocessing codes.
Will proceed without, but some convergence tests will not be available.


We define our input parameters. BCS is best for undetermined models and/or large basis sets.

In [ ]:
# PC parameters
nord = 8            # Order of the final PCE basis
ndim = 2            # Number of dimensions
pc_type = "LU"      # Polynomial type
pc_alpha = 0.0      # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
pc_beta = 1.0       # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs

# BCS parameters
niter = 3                                        # Number of iterations to run, must be > 0
eta = 1/np.power(10,[i for i in range(1,16)])    # Threshold for stopping the algorithm
ntry = 5                                         # Number of splits for cross-validation of the retained basis terms 
eta_folds = 5                       # Number of folds to use for eta cross-validation                          
mindex_growth = 'nonconservative'   # Method for basis growth; options are "conservative," "nonconservative," or None
regparams = None                    # Regularization weights (provide an array or value); if None, they are autopopulated
sigma = 1e-8                        # Inital noise variance; updated in BCS
trval_frac = None                   # Fraction of total data to use in each split
npccut = None                       # Maximum number of PC coefficients to retain
pcf_thr = None                      # Minimum value (by magnitude) for PC coefficients
verbose = 0                         # Flag for print statements

# Model Choice 
model = 'genz_osc'   # Choices are 'genz_osc', 'genz_exp', 'genz_cont','genz_gaus','genz_cpeak', 'genz_ppeak'

We randomly generate training and testing data.

In [ ]:
# PC model with a full basis set
pc_model =  uqtkpce.PCSet("NISPnoq", nord, ndim, pc_type, pc_alpha, pc_beta)

# Random number generator
rng = qmc.LatinHypercube(d=ndim, seed=42)

#Training
nTrain = int(pc_model.GetNumberPCTerms()*0.75) # Number of training samples
x_train = 2*rng.random(n=nTrain)-1
y_train = func(x_train, model, np.ones(ndim+1))

#Testing
nTest = 10000  # Number of testing samples
x_test = 2*rng.random(n=nTest)-1
y_test = func(x_test, model, np.ones(ndim+1))

We perform BCS.

In [ ]:
# PC object with the starting basis

# determine order of the starting basis
if mindex_growth == None:
    start_ord = nord # if no basis growth, order of the final basis = order of the starting basis
else:
    start_ord = nord-niter+1 # if basis growth, shrink starting basis to allow for growth niter-1 times
    
pc_start = uqtkpce.PCSet("NISPnoq", start_ord, ndim, pc_type, pc_alpha, pc_beta)

# Determine coefficients through BCS
pc_final, c_k = pce_tools.UQTkBCS(pc_start, x_train, y_train, niter, eta, ntry, eta_folds, mindex_growth, regparams, sigma, trval_frac, npccut, pcf_thr, verbose)

Finally, we calculate the error.

In [ ]:
# Evaluate the PCE
pce_evals = pce_tools.UQTkEvaluatePCE(pc_final, c_k, x_test)

# Calculate error metric
MSE = np.square(np.subtract(y_test, pce_evals)).mean()
NRMSE=math.sqrt(MSE)/np.linalg.norm(y_test)
print("The NRMS error between a", ndim, "-dimensional", model, "function and a BCS-based PC surrogate of \
order", nord, "is")
NRMSE

In [ ]:


n_components, n_features = 512, 100
n_nonzero_coefs = 17

# generate the data

# y = Xw
# y (n_features, n_samples)
# X (n_features, n_components)
# w (n_components, n_samples)
n_samples = 1
# |x|_0 = n_nonzero_coefs

y, X, w = make_sparse_coded_signal(
    n_samples=1,
    n_components=n_components,
    n_features=n_features,
    n_nonzero_coefs=n_nonzero_coefs,
    random_state=0,
    data_transposed=True,
)

(idx,) = w.nonzero()

# distort the clean signal
y_noisy = y + 0.05 * np.random.randn(len(y))

# plot the sparse signal
plt.figure(figsize=(7, 7))
plt.subplot(4, 1, 1)
plt.xlim(0, 512)
plt.title("Sparse signal")
plt.stem(idx, w[idx])

# plot the noise-free reconstruction
omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
omp.fit(X, y)
coef = omp.coef_
(idx_r,) = coef.nonzero()
plt.subplot(4, 1, 2)
plt.xlim(0, 512)
plt.title("Recovered signal from noise-free measurements")
plt.stem(idx_r, coef[idx_r])

# plot the noisy reconstruction
omp.fit(X, y_noisy)
coef = omp.coef_
(idx_r,) = coef.nonzero()
plt.subplot(4, 1, 3)
plt.xlim(0, 512)
plt.title("Recovered signal from noisy measurements")
plt.stem(idx_r, coef[idx_r])

# plot the noisy reconstruction with number of non-zeros set by CV
omp_cv = OrthogonalMatchingPursuitCV()
omp_cv.fit(X, y_noisy)
coef = omp_cv.coef_
(idx_r,) = coef.nonzero()
plt.subplot(4, 1, 4)
plt.xlim(0, 512)
plt.title("Recovered signal from noisy measurements with CV")
plt.stem(idx_r, coef[idx_r])

plt.subplots_adjust(0.06, 0.04, 0.94, 0.90, 0.20, 0.38)
plt.suptitle("Sparse signal recovery with Orthogonal Matching Pursuit", fontsize=16)
plt.show()

In [2]:
n_c = 512 # number of coefficients
n_ys = 100 # number of y samples
n_nz_c = 20 # number of nonzero coeffs

# generate the data

# ys = A coeffs
# ys (n_features, n_samples)
# A (n_features, n_components)
# coeffs (n_components, n_samples)
# n_samples = 1 (is more like a set of replicate problems)
# |x|_0 = n_nonzero_coefs

ys, A, coeffs = make_sparse_coded_signal(
    n_samples=1,
    n_components=n_c,
    n_features=n_ys,
    n_nonzero_coefs=n_nz_c,
    random_state=0,
    data_transposed=True,
)

(idc,) = coeffs.nonzero()

In [3]:
# add noise to the outputs
ys_noisy = ys + 0.05 * np.random.randn(len(ys))

In [4]:
# reconstruction of coefficients with OMP
omp_cv = OrthogonalMatchingPursuitCV()
omp_cv.fit(A, ys_noisy)
omp_coef = omp_cv.coef_
(idc_omp,) = omp_coef.nonzero()

In [5]:
omp_coef[idc_omp]

array([ 0.19342191, -0.42948835, -0.38090366, -0.3526881 ,  1.59331601,
       -0.89616794,  1.63950051, -2.75911696, -1.48202232, -1.44564498,
        1.44932224, -1.07117147,  0.21082588, -3.34853889, -0.28891701,
       -0.256209  ])

In [10]:
# reconstruction of coefficients with BCS
reg_params = np.array([])
bcs_coef = pce_tools.UQTkCallBCSDirect(A,ys_noisy,0.05,1.e-9,reg_params,verbose=True)

BCS has selected 15 basis terms out of 512 basis terms


In [11]:
omp_coef.shape

(512,)

In [12]:
idc_omp

array([ 46,  68,  88, 158, 195, 226, 268, 269, 339, 355, 359, 399, 467,
       492, 494, 509])

In [13]:
(idc_bcs,) = bcs_coef.nonzero()
idc_bcs

array([ 68,  88, 158, 195, 226, 268, 269, 339, 355, 359, 399, 467, 492,
       494, 509])

In [14]:
bcs_coef[idc_bcs]

array([-0.23975996, -0.31060015, -0.05457724,  1.58877797, -0.87092082,
        1.55721957, -2.66490749, -1.42951065, -1.40130787,  1.38692746,
       -1.0684128 ,  0.06093616, -3.26011169, -0.1083081 , -0.15859183])

In [15]:
bcs_coef[100]

0.0